In [ ]:
import torch

### Scripting

In [ ]:
def my_function(x):
    if x.mean() > 1.0:
        r = torch.tensor(1.0)
    else:
        r = torch.tensor(2.0)
    return r

In [ ]:
fscript = torch.jit.script(my_function)
fscript.graph

graph(%x.1 : Tensor):
  %10 : bool = prim::Constant[value=0]()
  %2 : NoneType = prim::Constant()
  %4 : float = prim::Constant[value=1.]() # <ipython-input-3-e20642937d0c>:2:18
  %12 : float = prim::Constant[value=2.]() # <ipython-input-3-e20642937d0c>:5:25
  %3 : Tensor = aten::mean(%x.1, %2) # <ipython-input-3-e20642937d0c>:2:7
  %5 : Tensor = aten::gt(%3, %4) # <ipython-input-3-e20642937d0c>:2:7
  %7 : bool = aten::Bool(%5) # <ipython-input-3-e20642937d0c>:2:7
  %r : Tensor = prim::If(%7) # <ipython-input-3-e20642937d0c>:2:4
    block0():
      %r.1 : Tensor = aten::tensor(%4, %2, %2, %10) # <ipython-input-3-e20642937d0c>:3:12
      -> (%r.1)
    block1():
      %r.3 : Tensor = aten::tensor(%12, %2, %2, %10) # <ipython-input-3-e20642937d0c>:5:12
      -> (%r.3)
  return (%r)

In [ ]:
fscript(torch.ones(2, 2))

tensor(2.)

### Tracing

In [ ]:
def my_function(x):
    if x.mean() > 1.0:
        r = torch.tensor(1.0)
    else:
        r = torch.tensor(2.0)
    return r

In [ ]:
ftrace = torch.jit.trace(my_function, torch.ones(2, 2))
ftrace.graph

<ipython-input-6-e20642937d0c>:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.mean() > 1.0:
<ipython-input-6-e20642937d0c>:5: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  r = torch.tensor(2.0)


graph(%x : Float(2, 2, strides=[2, 1], requires_grad=0, device=cpu)):
  %5 : Float(requires_grad=0, device=cpu) = prim::Constant[value={2}]() # <ipython-input-6-e20642937d0c>:5:0
  %6 : Device = prim::Constant[value="cpu"]() # <ipython-input-6-e20642937d0c>:5:0
  %7 : int = prim::Constant[value=6]() # <ipython-input-6-e20642937d0c>:5:0
  %8 : bool = prim::Constant[value=0]() # <ipython-input-6-e20642937d0c>:5:0
  %9 : bool = prim::Constant[value=0]() # <ipython-input-6-e20642937d0c>:5:0
  %10 : NoneType = prim::Constant()
  %11 : Float(requires_grad=0, device=cpu) = aten::to(%5, %6, %7, %8, %9, %10) # <ipython-input-6-e20642937d0c>:5:0
  %12 : Float(requires_grad=0, device=cpu) = aten::detach(%11) # <ipython-input-6-e20642937d0c>:5:0
  return (%12)

In [ ]:
ftrace(torch.ones(2, 2).add_(1.0))

tensor(2.)

In [ ]:
fscript(torch.ones(2, 2).add_(1.0))

tensor(1.)

In [ ]:
fscript.code

'def my_function(x: Tensor) -> Tensor:\n  if bool(torch.gt(torch.mean(x), 1.)):\n    r = torch.tensor(1.)\n  else:\n    r = torch.tensor(2.)\n  return r\n'

In [ ]:
ftrace.code

'def my_function(x: Tensor) -> Tensor:\n  _0 = torch.to(CONSTANTS.c0, torch.device("cpu"), 6)\n  return torch.detach(_0)\n'

In [ ]:
fscript.save("my_func.pt")

In [ ]:
!unzip my_func.pt

Archive:  my_func.pt
 extracting: my_func/data.pkl        
  inflating: my_func/code/__torch__.py  
  inflating: my_func/code/__torch__.py.debug_pkl  
 extracting: my_func/constants.pkl   
 extracting: my_func/version         


In [ ]:
ftrace.save("traced_my_func.pt")

In [ ]:
!unzip traced_my_func.pt

Archive:  traced_my_func.pt
 extracting: traced_my_func/data.pkl  
  inflating: traced_my_func/code/__torch__.py  
  inflating: traced_my_func/code/__torch__.py.debug_pkl  
 extracting: traced_my_func/constants/0  
 extracting: traced_my_func/constants.pkl  
 extracting: traced_my_func/version  
